# *Project Euler* 编程练习(1) 1~10

In [1]:
#基本库导入
%run BuiltInPackage
#第三方库导入
%run ThirdPartyPackage
#自定义库导入
import Fibonacci 
import NumberTheory

In [2]:
#一次性代码(暴力for循环法)
def pe_1_disposable_1(): 
    s = 0; 
    for n in range(1000): 
        if n % 3 == 0 or n % 5 == 0: 
            s += n; 
    return(s); 

In [3]:
#一次性代码(Filter法)
def pe_1_disposable_2(): 
    return(sum(filter(lambda n: not n % 3 or not n % 5, range(1000)))); 

In [4]:
#一次性代码(推导式法)
def pe_1_disposable_3(): 
    return(sum(n for n in range(1000) if not n % 3 or not n % 5)); 

In [5]:
#大操大办的代码(效率换可读性, 用类/静态方法模拟命名空间)
class ProjectEuler00001(): 
    
    #使用的系数 
    FACTORS = (3, 5, ); 
    
    #静态方法: 判定一个数num是否为若干个数factors中, 任一因子的整数倍
    @staticmethod
    def is_multiples_of(num: int, factors: list or tuple) -> bool: 
        return(any(not(num % factor) for factor in factors)); 
    
    #静态方法: 计算小于upper的所有正整数当中, 能被factors中的
    #一个以上的数整除的所有数之和
    #factor为可选参数, 缺省时, 为本题题干使用的系数, 即FACTORS. 
    @staticmethod
    def sum_of_all_multiples(upper: int, 
        factors: list or tuple = FACTORS) -> int: 
        #因数按照升序排列
        _factors = list(factors); 
        _factors.sort(); 
        result = sum(num 
            for num in range(1, upper) 
            if ProjectEuler00001.is_multiples_of(num, _factors)
        ); 
        return(result); 

In [6]:
#执行一次性代码
print(pe_1_disposable_1(), pe_1_disposable_2(), pe_1_disposable_3(), sep="\x20")

233168 233168 233168


In [7]:
#题干中的示例
ProjectEuler00001.sum_of_all_multiples(10) 

23

In [8]:
#本题的待求结果
ProjectEuler00001.sum_of_all_multiples(1000) 

233168

In [9]:
#扩展性测试: 1至100之间的所有偶数之和: 2+4+...+100
ProjectEuler00001.sum_of_all_multiples(101, (2, )) 

2550

In [10]:
#由于代码中封装了类, 因此存在额外的开销, 但有助于提升代码可读性和扩展性
%timeit -r 100 -n 100 pe_1_disposable_1()
%timeit -r 100 -n 100 pe_1_disposable_2()
%timeit -r 100 -n 100 pe_1_disposable_3()
%timeit -r 100 -n 100 ProjectEuler00001.sum_of_all_multiples(1000) 

144 µs ± 7.7 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
208 µs ± 2.79 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
142 µs ± 1.92 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
927 µs ± 9.71 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


***

## Problem 2: Even Fibonacci numbers

Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with 1 and 2, the first 10 terms will be:
$$1, 2, 3, 5, 8, 13, 21, 34, 55, 89, \ldots$$

By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.


背景知识: Fibonacci数列的计算公式
> 按照本题定义, Fibonacci数列的递推公式为
> $$Fib(n) =\begin{cases} Fib(n - 1) + Fib(n - 2)&, x \ge 2, \\ 1&, x = 0, 1 \end{cases}$$,
> 其中第$3 n + 2$项恒为偶数, 可由数学归纳法证明. 
>
> 该数列属于一阶线性递推数列, 利用特征方程法求得其通项公式为
> $$Fib(n) = {\frac {1} {\sqrt{5}} \left( \left( \frac {1 + \sqrt{5}} {2} \right)^{n + 1} - \left( \frac {1 - \sqrt{5}} {2} \right)^{n + 1}\right)},~n \in \mathbb{N}$$

In [11]:
#一次性代码(暴力while循环法, Fibonacci数列使用递推公式递归定义, 直接取第3n+2项)
#计算速度不忍直视, 因为数据没有复用
def pe_2_disposable_1(upper): 
    def fib(n): 
        if n <= 2: 
            return(n); 
        else: 
            return(fib(n - 1) + fib(n - 2)); 
    s = 0; i = 2; f = fib(i); 
    while f <= upper: 
        s += f; i += 3; f = fib(i); 
    return(s); 

In [12]:
#一次性代码(递归定义, 并建立缓存)
def pe_2_disposable_2(upper): 
    fib_ls = [1, 1, 2]; 
    def fib(n): 
        if n >= 3: 
            for i in range(len(fib_ls), n + 1): 
                fib_ls.insert(i, fib_ls[i - 1] + fib_ls[i - 2]);
    i = 2; 
    while fib_ls[-1] <= upper: 
        i += 3; fib(i); 
    s = tuple(fib_ls[2: : 3]); 
    return(sum(s[: -1])); 

In [13]:
#一次性代码(暴力while循环法, Fibonacci数列使用通项公式定义, 直接取第3n+2项)
#根号5直接用math, Decimal, np等包计算, 将出现误差
def pe_2_disposable_3(upper): 
    
    #不要使用sympy.sqrt代替math.sqrt求fib(n)通项, 因为速度极慢(sympy通用性的代价)
    SQRT_5 = math.sqrt(5); 
    PHI = (1 + SQRT_5) / 2; 
    PHI_MINOR = (1 - SQRT_5) / 2; 
    
    def fib(n): 
        f = (PHI ** (n + 1) - PHI_MINOR ** (n + 1)) / SQRT_5; 
        return(round(f)); 
    
    s = 0; i = 2; f = fib(i); 
    while f <= upper: 
        s += f; i += 3; f = fib(i); 
    return(s); 

In [14]:
#大操大办的代码(效率换可读性, 用类/静态方法模拟命名空间)
class ProjectEuler00002(): 
    
    #利用Fibonacci的通项公式计算,  计算过程中保留根号以避免浮点数的舍入误差
    #需要利用python面向对象的范式, 定义各种抽象代数
  
    #静态方法: 计算Fibonacci数列中, 不大于upper的所有偶数之和
    @staticmethod
    def sum_of_even_fibonacci(upper: int) -> int: 
        s = 0; idx = 2; 
        #FibonacciSequence对象的定义详见Fibonacci.py
        fib = Fibonacci.FibonacciSequence.general_term(idx); 
        while fib <= upper: 
            s += fib; idx += 3; 
            fib = Fibonacci.FibonacciSequence.general_term(idx); 
        return(s); 

In [15]:
#定义上限
pe_2_upper_1 = int(Decimal("4e+06")); #题干上限
pe_2_upper_2 = int(Decimal("4e+42")); #扩展性上限

In [16]:
#执行一次性代码
print(pe_2_disposable_1(pe_2_upper_1), 
    pe_2_disposable_2(pe_2_upper_1), 
    pe_2_disposable_3(pe_2_upper_1), 
    sep="\x20"
)

4613732 4613732 4613732


In [17]:
#本题的待求结果
ProjectEuler00002.sum_of_even_fibonacci(pe_2_upper_1)

2851443

In [18]:
#扩展性测试: 
#如果本题中的Fibonacci数上限再提升若干个数量级, pe_2_disposable_3将出现误差
print(
    pe_2_disposable_2(pe_2_upper_2), 
    pe_2_disposable_3(pe_2_upper_2), 
    sep="\n"
)
ProjectEuler00002.sum_of_even_fibonacci(pe_2_upper_2)

2517322709142507162883217709822239169909116
2517322709142524178949400468714733211118488


1555790994902035093049660322863084563868852

In [19]:
%timeit -r 5 -n 1 pe_2_disposable_1(pe_2_upper_1)
%timeit -r 100 -n 100 pe_2_disposable_2(pe_2_upper_1)
%timeit -r 100 -n 100 pe_2_disposable_3(pe_2_upper_1)
%timeit -r 25 -n 25 \
    ProjectEuler00002.sum_of_even_fibonacci(pe_2_upper_1)

4.13 s ± 3.01 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
22.5 µs ± 623 ns per loop (mean ± std. dev. of 100 runs, 100 loops each)
17.1 µs ± 1.67 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
16.3 ms ± 61.3 µs per loop (mean ± std. dev. of 25 runs, 25 loops each)


In [20]:
%timeit -r 100 -n 100 pe_2_disposable_2(pe_2_upper_2)
%timeit -r 5 -n 1 \
    ProjectEuler00002.sum_of_even_fibonacci(pe_2_upper_2)

137 µs ± 3.91 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
707 ms ± 2.83 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


***

## Problem 3: Largest prime factor

The prime factors of $13195$ are $5$, $7$, $13$ and $29$.

What is the largest prime factor of the number $600851475143$?


In [21]:
#一次性代码: 暴力试除法
def pe_3_disposable_1(): 
    n = 600851475143; x0 = 2; 
    last_prime = False; 
    while not last_prime: 
        last_prime = True; 
        for x in range(x0, int(math.sqrt(n))): 
            q, r = divmod(n, x); 
            if r == 0: 
                last_prime = False; 
                n = q; x0 = x; 
                break; 
    return(n); 

In [22]:
#大操大办的代码
class ProjectEuler00003(): 
    
    #静态方法: 质因数分解(暴力筛选法)
    @staticmethod
    def fectorize(num: int) -> list: 
        p = NumberTheory.Primes(); n = num; 
        p.sieve_of_eratosthenes(int(math.sqrt(n))); 
        primes = p.primes.keys(); divis = []; 
        last_prime = False; 
        while not last_prime: 
            last_prime = True; 
            for x in primes: 
                q, r = divmod(n, x); 
                if r == 0: 
                    last_prime = False; 
                    n = q; x0 = x; divis.append(x0); 
                    break; 
                elif x > q: 
                    continue
        return(divis); 

In [23]:
#题干中的示例
ProjectEuler00003.fectorize(13195)

[5, 7, 13, 29]

In [24]:
#本题的待求结果
ProjectEuler00003.fectorize(600851475143)

[71, 839, 1471, 6857]

In [25]:
#在分解质因数上, 通过筛选法获取质数用于分解的策略, 在效率上不具备优势
#原因是质数被存储在dict中, 建立过程较费时
%timeit -r 100 -n 100 pe_3_disposable_1()
%timeit -r 5 -n 1 ProjectEuler00003.fectorize(600851475143)

416 µs ± 3.19 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
229 ms ± 2.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


***

## Problem 4: Largest palindrome product

A palindromic number reads the same both ways. The largest palindrome made from the product of two $2$-digit numbers is $9009 = 91 \times 99$.

Find the largest palindrome made from the product of two $3$-digit numbers.

提示: 两个三位整数相乘最多是六位整数, 因此本题没有过多技术含量

In [26]:
#一次性代码
def pe_4_disposable_1(): 
    n = 999; 
    while n > 100: 
        ns = str(n); palin = int(ns + ns[: : -1]); 
        for f1 in range(999, int(math.sqrt(palin)), -1):
            f2, r = divmod(palin, f1); 
            if r == 0: 
                return(palin); 
        n -= 1; 

In [27]:
pe_4_disposable_1()

906609

***

## Problem 5: Smallest multiple

$2520$ is the smallest number that can be divided by each of the numbers from $1$ to $10$ without any remainder.

What is the smallest positive number that is evenly divisible by all of the numbers from $1$ to $20$?

提示: 计算正整数$1$至$n$的最小公倍数

In [28]:
#一次性代码: numpy二元LCM + reduce
def pe_5_disposable_1(upper): 
    #使用ndarray存储Fraction是为了防止计算较大的LCM时发生溢出, 得到错误结果
    return(reduce(
        np.lcm, 
        np.array(tuple(Fraction(x) for x in range(1, upper + 1)))
    ) ); 

In [29]:
#一次性代码: math二元GCD构造LCM + reduce
def pe_5_disposable_2(upper): 
    def lcm(x, y): 
        return(x * y // math.gcd(int(x), int(y))); 
    return(reduce(
        lcm, range(1, upper + 1)
    ) ); 

In [30]:
#大操大办的代码
class ProjectEuler00005(): 
    
    #静态方法
    @staticmethod
    def smallest_multiple(upper): 
    #直接对np.lcm调用reduce方法, 配合ndarray后, 会自动执行向量化
    #使用ndarray存储Fraction是为了防止计算较大的LCM时发生溢出, 得到错误结果
        return(np.lcm.reduce(
            np.array(tuple(Fraction(x) for x in range(1, upper + 1)))
        ) ); 

In [31]:
#定义上限
pe_5_upper_1 = 20; #题干上限
pe_5_upper_2 = 200; #扩展性上限

In [32]:
#执行一次性代码
print(
    pe_5_disposable_1(pe_5_upper_1), 
    pe_5_disposable_2(pe_5_upper_1), 
    sep="\x20"
)

232792560 232792560


In [33]:
#题干中的示例
int(ProjectEuler00005.smallest_multiple(10))

2520

In [34]:
#本题的待求结果
int(ProjectEuler00005.smallest_multiple(pe_5_upper_1))

232792560

In [35]:
#扩展性测试
print(
    pe_5_disposable_1(pe_5_upper_2), 
    pe_5_disposable_2(pe_5_upper_2), 
    sep="\n"
)
int(ProjectEuler00005.smallest_multiple(pe_5_upper_2))

337293588832926264639465766794841407432394382785157234228847021917234018060677390066992000
337293588832926264639465766794841407432394382785157234228847021917234018060677390066992000


337293588832926264639465766794841407432394382785157234228847021917234018060677390066992000

In [36]:
%timeit -r 20 -n 20 pe_5_disposable_1(pe_5_upper_1)
%timeit -r 20 -n 20 pe_5_disposable_2(pe_5_upper_1)
%timeit -r 20 -n 20 ProjectEuler00005.smallest_multiple(pe_5_upper_1)

1.31 ms ± 28.3 µs per loop (mean ± std. dev. of 20 runs, 20 loops each)
14.7 µs ± 4.09 µs per loop (mean ± std. dev. of 20 runs, 20 loops each)
1.06 ms ± 21.3 µs per loop (mean ± std. dev. of 20 runs, 20 loops each)


In [37]:
#numpy的代码为实现ndarray数组中元素的批量运算, 实现过特定优化. 
#代价是对单个元素执行速度较慢; 
#向量化在数组规模较大时方能体现效率, 注意: ndarray在存储变长数据上没有优势
%timeit -r 20 -n 20 pe_5_disposable_1(pe_5_upper_2)
%timeit -r 20 -n 20 pe_5_disposable_2(pe_5_upper_2)
%timeit -r 20 -n 20 ProjectEuler00005.smallest_multiple(pe_5_upper_2)

13.6 ms ± 93.3 µs per loop (mean ± std. dev. of 20 runs, 20 loops each)
184 µs ± 4.5 µs per loop (mean ± std. dev. of 20 runs, 20 loops each)
11 ms ± 61.6 µs per loop (mean ± std. dev. of 20 runs, 20 loops each)


## Problem 6: Sum square difference

The sum of the squares of the first ten natural numbers is,
$$1^2+2^2+\ldots +10^2=385$$

The square of the sum of the first ten natural numbers is,
$$(1+2+\ldots+10)^2 = 55^2 = 3025$$

Hence the difference between the sum of the squares of the first ten natural numbers and the square of the sum is $3025 − 385 = 2640$.

Find the difference between the sum of the squares of the first one hundred natural numbers and the square of the sum.

In [38]:
#一次性代码: 直接累加(使用np.array)
def pe_6_disposable_1(num): 
    seq = np.arange(1, num + 1, dtype=np.uint64); 
    #数值过大时, 使用Fraction防止溢出
    if num >= 65536: 
        seq = np.vectorize(Fraction)(seq); 
    seq_2 = seq ** 2; 
    diff = np.sum(seq) ** 2 - np.sum(seq_2); 
    return(int(diff)); 

In [39]:
#大操大办的代码
class ProjectEuler00006(): 
    
    #直接使用数列前n项和公式
    SUM_LIN_SEQ = np.polynomial.Polynomial((0, Fraction(1, 2), Fraction(1,2))); 
    SUM_QUAD_SEQ = np.polynomial.Polynomial(
        (0, Fraction(1, 6), Fraction(1, 2), Fraction(1, 3))
    ); 
    DIFF = SUM_LIN_SEQ ** 2 - SUM_QUAD_SEQ
    #虽然numpy的多项式系数可以Fraction形式存在, 但多项式求值过程中会转化为浮点小数, 引入误差
    #因此需要手动提取系数, 并编写秦九韶-Horner算法求解多项式的值
    DIFF_COEF = DIFF.coef; 
    
    @staticmethod
    def difference_of_sum(x: int) -> int: 
        poly = ProjectEuler00006.DIFF_COEF[-1]; 
        for coeff in ProjectEuler00006.DIFF_COEF[-2: None: -1]: 
            poly *= x; poly += coeff; 
        return(int(poly)); 

In [40]:
#定义上限
pe_6_upper_1 = 100; #题干上限
pe_6_upper_2 = 10 ** 4; #扩展性上限

In [41]:
#执行一次性代码
pe_6_disposable_1(pe_6_upper_1)

25164150

In [42]:
#题干中的示例
int(ProjectEuler00006.difference_of_sum(10))

2640

In [43]:
#本题的待求结果
int(ProjectEuler00006.difference_of_sum(pe_6_upper_1))

25164150

In [44]:
#扩展性测试
print(pe_6_disposable_1(pe_6_upper_2))
int(ProjectEuler00006.difference_of_sum(pe_6_upper_2))

2500166641665000


2500166641665000

In [45]:
%timeit -r 100 -n 100 pe_6_disposable_1(pe_6_upper_1)
%timeit -r 100 -n 100 ProjectEuler00006.difference_of_sum(pe_6_upper_1)

31.6 µs ± 4.89 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
41.1 µs ± 2.61 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [46]:
#在数据较大时, 求和公式更具优势
%timeit -r 100 -n 100 pe_6_disposable_1(pe_6_upper_2)
%timeit -r 100 -n 100 ProjectEuler00006.difference_of_sum(pe_6_upper_2)

67.2 µs ± 2.83 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
42.5 µs ± 1.22 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


## Problem 7: $10001$st prime
By listing the first six prime numbers: $2$, $3$, $5$, $7$, $11$, and $13$, we can see that the $6$th prime is $13$.

What is the $10001$st prime number?

背景知识: 素数分布近似公式: 
> 不大于$n$的素数数量$\varpi(n)$可取不足近似值
> $$\varpi(n) > \frac {n} {\mathrm{ln} n}$$
> 因此, 不大于$10^5$的素数至少有$\lceil {\frac {10^5} {\mathrm{ln} 10^5} \rceil} = 8686$个; 不大于$10^6$的素数至少有$\lceil {\frac {10^6} {\mathrm{ln} 10^6} \rceil} = 72383$个; 

In [47]:
#一次性代码: 暴力试除法
def pe_7_disposable_1(): 
    i = 1; n = 3; 
    while i < 10001: 
        is_pri = NumberTheory.Primes.prime_definition(n); 
        if is_pri: i += 1; 
        n += 2; 
    return(n - 2); 

In [48]:
#一次性代码: 使用Eratosthenes筛法
def pe_7_disposable_2(): 
    p = NumberTheory.Primes(); 
    p.sieve_of_eratosthenes(10 ** 6); 
    prime = list(p.primes.keys()); 
    return(prime[10000]); 

In [49]:
print(pe_7_disposable_1(), pe_7_disposable_2())

104743 104743


In [50]:
%timeit -r 5 -n 1 pe_7_disposable_1()
%timeit -r 5 -n 1 pe_7_disposable_2()

436 ms ± 5.74 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
275 ms ± 2.81 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


***

## Problem 8: Largest product in a series

The four adjacent digits in the $1000$-digit number that have the greatest product are $9 \times 9 \times 8 \times  9 = 5832$.

> 73167176531330624919225119674426574742355349194934 
96983520312774506326239578318016984801869478851843 \
... \
05886116467109405077541002256983155200055935729725 
71636269561882670428252483600823257530420752963450

Find the thirteen adjacent digits in the $1000$-digit number that have the greatest product. What is the value of this product?

In [51]:
#题干所示的一千个数码组成的字符串
pe_8_material_1 = \
    "73167176531330624919225119674426574742355349194934" + \
    "96983520312774506326239578318016984801869478851843" + \
    "85861560789112949495459501737958331952853208805511" + \
    "12540698747158523863050715693290963295227443043557" + \
    "66896648950445244523161731856403098711121722383113" + \
    "62229893423380308135336276614282806444486645238749" + \
    "30358907296290491560440772390713810515859307960866" + \
    "70172427121883998797908792274921901699720888093776" + \
    "65727333001053367881220235421809751254540594752243" + \
    "52584907711670556013604839586446706324415722155397" + \
    "53697817977846174064955149290862569321978468622482" + \
    "83972241375657056057490261407972968652414535100474" + \
    "82166370484403199890008895243450658541227588666881" + \
    "16427171479924442928230863465674813919123162824586" + \
    "17866458359124566529476545682848912883142607690042" + \
    "24219022671055626321111109370544217506941658960408" + \
    "07198403850962455444362981230987879927244284909188" + \
    "84580156166097919133875499200524063689912560717606" + \
    "05886116467109405077541002256983155200055935729725" + \
    "71636269561882670428252483600823257530420752963450"; 

In [52]:
#一次性代码: 直接从字符串提取连续子串
def pe_8_disposable_1(digit): 
    def sub_str_to_prod(s): 
        return(reduce(oper.mul, tuple(int(char) for char in s))); 
    prods = tuple(sub_str_to_prod(pe_8_material_1[i: i + digit]) 
        for i in range(0, len(pe_8_material_1) - digit)
    ); 
    return(max(prods)); 

In [53]:
#一次性代码: numpy向量化方式处理二维定长数组
def pe_8_disposable_2(digit): 
    arr_sub_str = np.array(tuple(tuple(pe_8_material_1[i: -digit + i])
        for i in range(digit)
    )).astype(np.uint64); 
    prods = np.prod(arr_sub_str, axis=0); 
    return(max(prods)); 

In [54]:
#一次性代码: 使用向量化机制将字符串的内容转移至数组
def pe_8_disposable_3(digit): 
    arr_sub_str = np.zeros((digit, len(pe_8_material_1)), dtype=np.uint64); 
    arr_sub_str[0, : ] = np.array(tuple(pe_8_material_1)
        ).reshape(1, len(pe_8_material_1)); 
    for row in range(1, digit): 
        arr_sub_str[row, : ] = np.roll(arr_sub_str[0, : ], -row); 
    prods = np.prod(arr_sub_str[: , : -row], axis=0); 
    return(max(prods)); 

In [55]:
#大操大办的代码
class ProjectEuler00008(): 
    
    #静态方法: 计算字符串中所有数字的乘积
    @staticmethod
    def product_sub_str(q: collections.deque) -> int: 
        prod = reduce(oper.mul, q); 
        return(prod); 
    
    #静态方法: 求连续n个数的最大乘积
    @staticmethod
    def max_prod_sub_str(n: int) -> int: 
        #在RAM上创建字符数据流, 用于无重复地顺序读取字符
        strm = np.array(tuple(pe_8_material_1)).astype(np.int64); 
        #创建一个与需要计算乘积的子串等长的队列
        queue = collections.deque(list(), n); 
        #读取字符串前n个字符, 计算其中数字乘积, 初始化堆
        #heapq构建的堆是小根堆, 如需求最大值, 需要将入堆的元素取相反数
        queue.extend(strm[0: n]); 
        prod = -ProjectEuler00008.product_sub_str(queue); 
        heap = [prod]; 
        #每次从数据流读取一个字符, 将其加入队列尾(同时, 队头对应数目的字符丢弃)
        for char_new in strm[n: ]: 
            queue.append(char_new); 
            prod = -ProjectEuler00008.product_sub_str(queue); 
            heapq.heappush(heap, prod); 
        #读取完成后取出最大的乘积
        prod = -heap[0]; 
        return(prod); 

In [56]:
#执行一次性代码
print(
    pe_8_disposable_1(13), 
    pe_8_disposable_2(13), 
    pe_8_disposable_3(13), 
    sep="\x20"
)

23514624000 23514624000 23514624000


In [57]:
#题干中的示例
ProjectEuler00008.max_prod_sub_str(4)

5832

In [58]:
#本题的待求结果
ProjectEuler00008.max_prod_sub_str(13)

23514624000

In [59]:
#空间换时间
%timeit -r 50 -n 50 pe_8_disposable_1(13)
%timeit -r 50 -n 50 pe_8_disposable_2(13)
%timeit -r 50 -n 50 pe_8_disposable_3(13)
%timeit -r 50 -n 50 ProjectEuler00008.max_prod_sub_str(13)

6.35 ms ± 33.8 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)
8.23 ms ± 35.1 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)
1.11 ms ± 18.5 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)
2.85 ms ± 21.8 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)


***

# Problem 9: Special Pythagorean triplet

A Pythagorean triplet is a set of three natural numbers, $a<b<c$, for which,
$$a^2+b^2=c^2$$

For example, $3^2+4^2=9+16=25=5^2$.

There exists exactly one Pythagorean triplet for which $a + b + c = 1000$. Find the product $abc$.

提示: 勾股数$a, b, c ~ (a<b<c)$构成了直角三角形的三边长

In [60]:
#一次性代码: 暴力枚举
def pe_9_disposable_1(circ): 
    #提高枚举效率的方法: 考虑约束条件: a + b + c = 1000; a < b < c; a + b > c; 
    max_a = np.floor(circ / 3); 
    for a in np.arange(1, max_a + 1, dtype=np.uint64): 
        max_b_lt_c = np.floor((circ - a) / 2); 
        min_b_triangle = np.ceil(circ / 2) - a; 
        for b in np.arange(max(a, min_b_triangle), max_b_lt_c, 
            dtype=np.uint64
        ): 
            c = circ - a - b; 
            if a ** 2 + b ** 2 == c ** 2: 
                return(int(a * b * c)); 

In [61]:
#一次性代码: 方程组消元
def pe_9_disposable_2(circ): 
    #提高枚举效率的方法: 考虑约束条件: a + b + c = 1000; a < b < c; 
    max_a = np.floor(circ / 3); 
    for a in np.arange(1, max_a + 1, dtype=np.uint64): 
        #联立方程后消去c, 得到a与b之间的等式; 
        #用a反表示b, 要求b是整数, 因此divmod的商为b, 余数为0. 
        b, r = np.divmod(circ ** 2 / 2 - circ * a, circ - a); 
        if r == 0: 
            c = circ - a - b; 
            return(int(a * b * c)); 

In [62]:
print(pe_9_disposable_1(1000), pe_9_disposable_2(1000), )

31875000 31875000


In [63]:
%timeit -r 10 -n 10 pe_9_disposable_1(1000)
%timeit -r 10 -n 10 pe_9_disposable_2(1000)

152 ms ± 315 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
3.34 ms ± 76.1 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


***

## Problem 10: Summation of primes

The sum of the primes below $10$ is $2 + 3 + 5 + 7 = 17$.

Find the sum of all the primes below two million.

In [64]:
#一次性代码: 使用Eratosthenes筛法
#问题7的代码可以在此处复用, 因此, 筛法的代码位于自定义库中
def pe_10_disposable_1(upper): 
    p = NumberTheory.Primes(); 
    p.sieve_of_eratosthenes(upper); 
    total = sum(p.primes.keys()); 
    return(total); 

In [65]:
pe_10_disposable_1(2 * 10 ** 6)

142913828922

In [66]:
%timeit -r 5 -n 1 pe_10_disposable_1(2 * 10 ** 6)

546 ms ± 2.35 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
